In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import gzip
import random
import warnings
warnings.filterwarnings('ignore')

In [ ]:
n = 40428967  #total number of records in the clickstream data 
sample_size = 1000000
skip_values = sorted(random.sample(range(1,n), n-sample_size))

parse_date = lambda val : pd.datetime.strptime(val, '%y%m%d%H')
with gzip.open('../input/avazu-ctr-prediction/train.gz') as f:
    train = pd.read_csv(f,skiprows = skip_values)
train['hour'] = pd.to_datetime(train['hour'],format = '%y%m%d%H')    

In [ ]:
train_data=train.copy()
y_label=train_data.pop('click')
train_data=train_data.drop(['hour','id'],axis=1)

In [ ]:
# Get categorical columns
categorical_cols = train_data.select_dtypes(
  include = ['object']).columns.tolist()
print("Categorical columns: ")
print(categorical_cols)

# Iterate over categorical columns and apply hash function


In [ ]:
for col in categorical_cols:
	train_data[col] = train_data[col].apply(lambda x: hash(x))

In [ ]:
train_data.head()

In [ ]:
from sklearn.model_selection import train_test_split
#X = train_data.drop(['id', 'device_id', 'device_ip', 'hour', 'click'], axis=1)
#y = train_data.click
x_train,x_test,y_train,y_test = train_test_split(train_data,y_label,test_size = 0.2,random_state=21)

In [ ]:
from sklearn.preprocessing import StandardScaler
sclr = StandardScaler()
x_train.iloc[:,2:11] = sclr.fit_transform(x_train.iloc[:,2:11])
x_test.iloc[:,2:11] = sclr.transform(x_test.iloc[:,2:11])

In [ ]:
train_data['user_info']=train_data.device_ip+train_data.device_model+train_data.device_id
train_data=train_data.drop(['device_id','device_ip','device_model'],axis=1)
train_data['devtry']=train_data.device_type+train_data.banner_pos+train_data.device_conn_type
train_data=train_data.drop(['banner_pos','device_conn_type','device_type'],axis=1)

In [ ]:
import xgboost as xgb
from sklearn.decomposition import PCA
from sklearn.metrics import roc_curve,auc,confusion_matrix,precision_score,recall_score,roc_auc_score
# intialize pca and logistic regression model
pca = PCA(n_components=2)
params = {
    "objective": "binary:logistic",
    "booster" : "gbtree",
    "eval_metric": "logloss",
    "eta":0.1,
    "max_depth": 8,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "silent": 1,
}
xgclf=xgb.XGBClassifier(**params)

# fit and transform data
X_train_pca = pca.fit_transform(x_train)
X_test_pca = pca.transform(x_test)
xgclf.fit(X_train_pca, y_train,
        eval_set=[(X_test_pca, y_test)],
        eval_metric='logloss',
        verbose=False)
pca.explained_variance_ratio_


In [ ]:
from sklearn import metrics 
xgpred=xgclf.predict_proba(X_test_pca)
#fpr, tpr, thresholds = roc_curve(y_test, y_score[:, 1])
roc_auc_score = metrics.roc_auc_score(y_test,xgpred[:,1])
print(roc_auc_score)
print(xgclf.score(X_test_pca,y_test))

In [ ]:
from sklearn.feature_selection import mutual_info_classif
# determine the mutual information
mutual_info = mutual_info_classif(train_data,y_label,discrete_features=True)
mutual_info = pd.Series(mutual_info)
mutual_info.index = train_data.columns
mutual_info.sort_values(ascending=False)

In [ ]:
import xgboost as xgb
from sklearn.metrics import roc_curve,auc,confusion_matrix,precision_score,recall_score,roc_auc_score
params = {
    "objective": "binary:logistic",
    "booster" : "gbtree",
    "eval_metric": "logloss",
    "eta":0.1,
    "max_depth": 8,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "silent": 1,
}
xgclf=xgb.XGBClassifier(**params)
xgclf.fit(x_train, y_train,
        eval_set=[(x_test, y_test)],
        eval_metric='logloss',
        verbose=False)
xgpred=xgclf.predict_proba(x_test)

In [ ]:
from sklearn import metrics 
#y_score = xgclf.predict_proba(x_test)
#fpr, tpr, thresholds = roc_curve(y_test, y_score[:, 1])
roc_auc_score = metrics.roc_auc_score(y_test,xgpred[:,1])
print(roc_auc_score)
print(xgclf.score(x_test,y_test))

In [ ]:
from sklearn.model_selection import KFold,cross_val_score
# Set up k-fold
k_fold = KFold(n_splits = 5)

# Evaluate precision and recall for each fold
precision = cross_val_score(
  xgclf, x_train, y_train, cv = k_fold, scoring = 'precision_weighted')
recall = cross_val_score(
  xgclf, x_train, y_train, cv = k_fold, scoring = 'recall_weighted')
print("Precision scores: %s" %(precision)) 
print("Recall scores: %s" %(recall))
print(k_fold)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Exploratory Data Analysis


In [ ]:
print(train.columns)
print(train.dtypes)

## For missing values

In [ ]:
print(train.info())
print(train.isnull().sum().sum())

There are no missing values in the dataset.

## id

In [ ]:
train.drop('id',axis=1,inplace = True)

## Timestamp - Hour

In [ ]:
train['day_week'] = train['hour'].apply(lambda x: x.dayofweek)
train['hour_day'] = train['hour'].apply(lambda x: x.hour)

In [ ]:
train.groupby(['day_week','click']).size().unstack().plot(kind='bar')
plt.title('Days of the week')
train.groupby(['hour_day','click']).size().unstack().plot(kind='bar')
plt.title('Hours of a day')

In [ ]:
train.groupby('hour_day')['click'].sum().plot.bar(figsize=(12,6))
plt.ylabel('Number of clicks')
plt.title('Number of clicks by hour of day')
plt.show()

## Click

In [ ]:
sns.countplot(x='click',data=train)
plt.show()
print(train.click.value_counts()/len(train))

Nearly 17% is the CTR of the data provided.

## Banner Position

In [ ]:
train.groupby(['banner_pos', 'click']).size().unstack().plot(kind='barh')
train.groupby(['banner_pos','click']).size().unstack().iloc[2:,:].plot(kind='barh')

In [ ]:
site_features = ['site_id', 'site_domain', 'site_category']
train[site_features].describe()

## Hashing


In [ ]:
# Get categorical columns
categorical_cols = train.select_dtypes(
  include = ['object']).columns.tolist()
print("Categorical columns: ")
print(categorical_cols)

# Iterate over categorical columns and apply hash function
for col in categorical_cols:
	train[col] = train[col].apply(lambda x: hash(x))

## Feature Selection

In [ ]:
ad_corr = train.corr(method = 'pearson')
fig,ax = plt.subplots(figsize = (20,10))
sns.heatmap(ad_corr,
            xticklabels=ad_corr.columns,
            yticklabels=ad_corr.columns,
            cmap='RdBu_r',
            annot=True,
            linewidth=1,ax=ax)

We can remove the features which are irrelavant to the data, and not useful.

In [ ]:
from sklearn.model_selection import train_test_split
X = train.drop(['device_id','device_ip','hour','click'],axis=1)
y = train.click
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state=21)

In [ ]:
x_train.head()

In [ ]:
from sklearn.preprocessing import StandardScaler
sclr = StandardScaler()
x_train.iloc[:,8:15] = sclr.fit_transform(x_train.iloc[:,8:15])
x_test.iloc[:,8:15] = sclr.transform(x_test.iloc[:,8:15])

## Model

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
clf = DecisionTreeClassifier(max_depth=10)
clf.fit(x_train,y_train)

In [ ]:
print('Train Score:',clf.score(x_train,y_train))
print('Test Score:',clf.score(x_test,y_test))

In [ ]:
from sklearn.metrics import roc_curve,auc,confusion_matrix,precision_score,recall_score,roc_auc_score
y_score = clf.predict_proba(x_test)
fpr, tpr, thresholds = roc_curve(y_test, y_score[:, 1])
roc_auc_score = roc_auc_score(y_test,y_score[:,1])
print(roc_auc_score)

In [ ]:
random_probs = [0 for i in range(len(y_test))]
p_fpr, p_tpr, _ = roc_curve(y_test, random_probs, pos_label=1)

In [ ]:
plt.style.use('seaborn')
plt.plot(fpr,tpr,linestyle = '--',color = 'green',label='Decision Tree')
plt.plot(p_fpr,p_tpr,linestyle='--',color = 'blue')
plt.legend()
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('AUC-ROC Curve')

In [ ]:
y_pred = clf.predict(x_test)
precision = precision_score(y_test, y_pred, average = 'weighted')
recall = recall_score(y_test, y_pred, average = 'weighted')
print("Precision: %s, Recall: %s" %(precision, recall))

In [ ]:
matrix = confusion_matrix(y_test,y_pred)
tn, fp, fn, tp = matrix.ravel()
print(matrix)

In [ ]:
#For example:
#Here total_return and cost values are examples
total_return = 0.2
cost = 0.05
total_return = tp * total_return
total_cost = (tp + fp) * cost 
roi = total_return / total_cost
print("Total return: %s, Total cost: %s, ROI: %s" %(
  total_return, total_cost, roi))

In [ ]:
for max_depth_val in [2, 3, 5, 10, 15, 20]:
    clf = DecisionTreeClassifier(max_depth = max_depth_val)
    print("Evaluating tree with max_depth = %s" %(max_depth_val))
    y_pred = clf.fit(x_train,y_train).predict(x_test) 
    print("Confusion matrix: ")
    print(confusion_matrix(y_test, y_pred))
    prec = precision_score(y_test, y_pred, average = 'weighted')
    recall = recall_score(y_test, y_pred, average = 'weighted')
    print("Precision: %s, Recall: %s" %(prec, recall))

In [ ]:
from sklearn.model_selection import KFold,cross_val_score
# Set up k-fold
k_fold = KFold(n_splits = 5)

# Evaluate precision and recall for each fold
precision = cross_val_score(
  clf, x_train, y_train, cv = k_fold, scoring = 'precision_weighted')
recall = cross_val_score(
  clf, x_train, y_train, cv = k_fold, scoring = 'recall_weighted')
print("Precision scores: %s" %(precision)) 
print("Recall scores: %s" %(recall))
print(k_fold)

In [ ]:
for max_depth_val in [3, 5, 10]:
    k_fold = KFold(n_splits = 4)
    clf = DecisionTreeClassifier(max_depth = max_depth_val)
    print("Evaluating Decision Tree for max_depth = %s" %(max_depth_val))
    y_pred = clf.fit(x_train, y_train).predict(x_test) 
  
    cv_precision = cross_val_score(clf, x_train, y_train, cv = k_fold, scoring = 'recall_weighted')
    precision = recall_score(y_test, y_pred, average = 'weighted')
    print("Cross validation Recall: %s" %(cv_precision))
    print("Test Recall: %s" %(precision))
  

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

param_grid = {
    'bootstrap': [True],
    'max_depth': [10], 
    'n_estimators':[2,5,10,20,50],
    'min_samples_split': [2, 3, 4],
    'max_features' : ['log2']
}

rf = RandomForestClassifier(random_state=42)

rf_search = RandomizedSearchCV(estimator = rf, param_distributions=param_grid, 
                          cv = 5, n_jobs = -1, verbose = 2, n_iter = 10)

model = rf_search.fit(x_train,y_train)

In [ ]:
rf_search.best_params_

In [ ]:
clf_rf = RandomForestClassifier(n_estimators= 50,
 min_samples_split= 2,
 max_features= 'log2',
 max_depth= 10,
 bootstrap= True,
 random_state=42)
clf_rf.fit(x_train,y_train)

In [ ]:
y_test_pred = clf_rf.predict(x_test)

print(confusion_matrix(y_test, y_test_pred))
prec = precision_score(y_test, y_test_pred, average = 'weighted')
recall = recall_score(y_test, y_test_pred, average = 'weighted')
print("Precision: %s, Recall: %s" %(prec, recall))

In [ ]:
from sklearn.tree import export_graphviz
from six import StringIO
import pydot as pydot
import graphviz
from IPython.display import Image

dot_data = export_graphviz(clf,out_file=None,filled=True,rounded=True,special_characters=True)
graph = graphviz.Source(dot_data)
graph
